In [25]:
# 4a
using Dates

In [36]:
# First, sort the lines in the file 
fmt = DateFormat("Y-m-d H:M")
lines = readlines("4-input.txt")
n = length(lines)

datetimes = Array{DateTime,1}(undef, n)
minutes = Array{Int64, 1}(undef, n)

for (i,l) in enumerate(lines)
    datetimes[i] = DateTime(l[2:17], fmt)
    minutes[i] = parse(Int64, l[16:17])
end
p = sortperm(datetimes)
sorted_lines = lines[p]
sorted_minutes = minutes[p]
sorted_lines[1:6]

6-element Array{String,1}:
 "[1518-02-24 23:57] Guard #1913 begins shift"
 "[1518-02-25 00:17] falls asleep"            
 "[1518-02-25 00:29] wakes up"                
 "[1518-02-25 00:54] falls asleep"            
 "[1518-02-25 00:56] wakes up"                
 "[1518-02-26 00:03] Guard #3469 begins shift"

In [64]:
# Create a dict of guards with the ID of the guards, 
# store the total minutes slept in it
d = Dict{Int64, Int64}()
r = r"#([0-9]+) "
i = 1
while i<length(sorted_lines)
    # We are at a guard ID line
    m = match(r, sorted_lines[i])
    id = parse(Int64, m.captures[1])
    while (i<length(sorted_lines))&&(!occursin(r, sorted_lines[i+1]))
        sleep_min = sorted_minutes[i+2] - sorted_minutes[i+1] -1
        haskey(d, id) ? d[id] = d[id] + sleep_min : d[id] = sleep_min
        i = i + 2
    end
    i = i + 1
end
# Reverse the dict to look up the max value
dr = Dict(value => key for (key, value) in d)
max_min = sort(collect(values(d)))[end]
lazy_guard_id = dr[max_min]
(lazy_guard_id, max_min)

(523, 484)

In [73]:
# Create a list of the 60 minutes and how often that specific guard was
# asleep then populate it
hour = collect(zeros(Int64, 60))
i = 1
while i<length(sorted_lines)
    # We are at a guard ID line
    m = match(r, sorted_lines[i])
    id = parse(Int64, m.captures[1])
    while (i<length(sorted_lines))&&(!occursin(r, sorted_lines[i+1]))
        if id == lazy_guard_id
            hour[(sorted_minutes[i+1]+1):(sorted_minutes[i+2])] = 
                hour[(sorted_minutes[i+1]+1):(sorted_minutes[i+2])] .+ 1
        end
        i = i + 2
    end
    i = i + 1
end
# Minutes 00 is the 1st element in the array
(sleeps, minute) = findmax(hour)
minute = minute - 1

38

In [74]:
# ID * minutes is the answer for part a
lazy_guard_id * minute

19874

In [81]:
# 4b
# Now I pay the price for not calculating the full thing the first time 
d = Dict{Int64, Array{Int64,1}}()
i = 1
while i<length(sorted_lines)
    # We are at a guard ID line
    m = match(r, sorted_lines[i])
    id = parse(Int64, m.captures[1])
    while (i<length(sorted_lines))&&(!occursin(r, sorted_lines[i+1]))
        if !haskey(d, id)
            d[id] = collect(zeros(Int64, 60))
        end
        d[id][(sorted_minutes[i+1]+1):(sorted_minutes[i+2])] = 
            d[id][(sorted_minutes[i+1]+1):(sorted_minutes[i+2])] .+ 1
        i = i + 2
    end
    i = i + 1
end

# Make a dict to hold the sleepiest minute and the number of sleeps
dm = Dict{Int64, Pair{Int64, Int64}}()
for (k, v) in d
    dm[k] = Pair(findmax(v)...)
end
dm

Dict{Int64,Pair{Int64,Int64}} with 20 entries:
  1069 => 11=>37
  1861 => 8=>39
  1399 => 12=>41
  1699 => 13=>37
  3253 => 6=>12
  2557 => 10=>25
  151  => 12=>45
  3469 => 9=>45
  1123 => 10=>30
  1759 => 13=>48
  389  => 8=>27
  73   => 6=>21
  2333 => 7=>33
  829  => 13=>48
  463  => 20=>50
  3463 => 14=>24
  653  => 6=>37
  523  => 14=>39
  1913 => 4=>18
  1597 => 7=>29

In [83]:
# Eyeball the above, sleepiest guard is #463 at minute 50-1 = 49
463*49

22687